In this lab we turn a pre-trained BERT model into a trainable Keras layer and apply it to the Aspect-Based Sentiment Analysis (ABSA). You can find the task description from (https://aclanthology.org/D19-1654.pdf).
This task provides a review text dataset with aspects.
Given a review and an aspect, we need to classify the sentiment conveyed towards this aspect on a three-point scale: POSITIVE, NEUTRAL or NEGATIVE.
This is a multi-class classification task.

BERT (Bidirectional Embedding Representations from Transformers) is a model for pre-training language representations that obtains state-of-the-art results on many NLP tasks. We demonstrate how to integrate BERT as a custom Keras layer using the Huggingface library.

In this lab, you will learn:

1) How to use the Huggingface library.

2) How to integrate BERT in the models built previously.

3) How to use the TPU from Colab. **Note**: Running BERT on the CPU will be very slow. Thus we recommend you to do this lab on a Colab TPU provided by Google.

# Lab 3 - Aspect-Based Sentiment Analysis with BERT

In [1]:
import keras
import numpy as np
from keras.layers import Lambda, GlobalAveragePooling1D, Dense, Embedding
from keras import backend as K
from keras.models import Sequential
import matplotlib.pyplot as plt

from keras.layers import LSTM, RNN, Dropout, Input, LeakyReLU, Bidirectional,Conv1D, GlobalMaxPooling1D
from keras.layers import Dense
from keras.models import Model


We first need to install the Huggingface's Transformers package. You can find the relevant doc from [here](https://huggingface.co/transformers/index.html).

In [2]:
pip install transformers

## Preprocessing and Tokenization

In this lab we will use DistilBERT (https://arxiv.org/pdf/1910.01108.pdf) instead of BERT: DistilBERT is a smaller and faster Transformer model trained via distilling BERT. It has 40% less parameters than the bert-base-uncased model. It runs 60% faster, while preserving 97% of BERT’s performance as measured on the GLUE language understanding benchmark.

It is easy to switch between DistilBERT and BERT using the Huggingface transformers package. This package provides many pre-trained and pre-built models that are easy to use via a few lines of code.

Before using DistilBERT or BERT, we need a tokenizer. Generally speaking, every BERT related model has its own tokenizer, trained for this particular model.
We can get the DistilBERT tokenizer from the **DistilBertTokenizer.from_pretrained** method.

In [3]:
from transformers import DistilBertTokenizer, RobertaTokenizer
import tqdm
distil_bert = 'distilbert-base-uncased' # Pick a pre-trained model

# Defining DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert, do_lower_case=True, add_special_tokens=True,
                                                max_length=128, pad_to_max_length=True)

def tokenize(sentences, tokenizer, pad_length=128, pad_to_max_length=True ):
    if type(sentences) == str:
        inputs = tokenizer.encode_plus(sentences, add_special_tokens=True, max_length=pad_length, pad_to_max_length=pad_to_max_length,
                                             return_attention_mask=True, return_token_type_ids=True)
        return np.asarray(inputs['input_ids'], dtype='int32'), np.asarray(inputs['attention_mask'], dtype='int32'), np.asarray(inputs['token_type_ids'], dtype='int32')

    input_ids, input_masks, input_segments = [],[],[]
    for sentence in sentences:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=pad_length, pad_to_max_length=pad_to_max_length,
                                             return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])

    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [4]:
inputs = tokenizer.tokenize("The capital of France is [MASK].")
print(inputs,'\n')

inputs = tokenizer.tokenize("This is a pretrained model.")
print(inputs,'\n')

ids,masks,segments = tokenize("The capital of France is [MASK].", tokenizer)
print(ids)
print(masks)
print(tokenizer.convert_ids_to_tokens(ids),"\n")

ids,masks,segments = tokenize("The capital of France is [MASK].", tokenizer, pad_to_max_length=False)
print(ids)
print(masks)
print(tokenizer.convert_ids_to_tokens(ids),"\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['the', 'capital', 'of', 'france', 'is', '[MASK]', '.'] 

['this', 'is', 'a', 'pre', '##train', '##ed', 'model', '.'] 

[ 101 1996 3007 1997 2605 2003  103 1012  102    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Then we can use this tokenizer to tokenize our data. When working with word2vec and GloVe, you tokenized sentences into words yourself and then converted the tokens into GloVe indices. But in BERT, we must use the BERT tokenizer that uses sub-word tokens.

For example, for the sentence: **This is a pretrained model.** our previous word-based tokenizer will generate the following tokens:

**"this", "is", "a", "pretrained", "model", "."**

Then you will find out that the token "pretrained" is not in the GloVe word dictionary. Thus we can not assign it a trained word vector.

The BERT tokenizer will separate the word "pretrained" into three sub-word tokens:

**'pre', '##train', '##ed'**

BERT thus uses these three token vectors to represent the word "pretrained". You will also see that the BERT tokenizer adds the special [CLS] token and the sentence separator [SEP] token.

## Loading Data

### Downloading and preprocessing data

Unlike the IMDB dataset that is included and preprocessed by Keras, the dataset we will be using is the aspect-based sentiment analysis (ABSA)  dataset, which consists of 5,297 labeled reviews. These are split into 4,297 reviews for training and 500 reviews for testing and validation, respectively.

For ABSA, sentiment polarities were assigned with respect to the aspect terms.  The start and end positions for each aspect term are provided.


In [5]:
import requests
def downloadfile(url):
  rq = requests.get(url)
  open(url.split('/')[-1], 'wb').write(rq.content)
downloadfile('https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data/MAMS-ATSA/raw/train.xml')
downloadfile('https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data/MAMS-ATSA/raw/val.xml')
downloadfile('https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data/MAMS-ATSA/raw/test.xml')


# The code is modified from https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data_process/utils.py
from xml.etree.ElementTree import parse

def parse_sentence_term(path, lowercase=False):
    tree = parse(path)
    sentences = tree.getroot()
    data = []
    split_char = '__split__'
    for sentence in sentences:
        text = sentence.find('text')
        if text is None:
            continue
        text = text.text
        if lowercase:
            text = text.lower()
        aspectTerms = sentence.find('aspectTerms')
        if aspectTerms is None:
            continue
        for aspectTerm in aspectTerms:
            term = aspectTerm.get('term')
            if lowercase:
                term = term.lower()
            polarity = aspectTerm.get('polarity')
            start = aspectTerm.get('from')
            end = aspectTerm.get('to')
            piece = [text , term,  polarity , start , end]
            data.append(piece)
    return data
train = parse_sentence_term("train.xml",True)
dev = parse_sentence_term("val.xml",True)
test = parse_sentence_term("test.xml",True)

print("Training entries: {}".format(len(train)))
print("Test entries: {}".format(len(test)))


Training entries: 11186
Test entries: 1336


Let’s first see some examples from the data:

In [6]:
print("SENTENCE \t ASPECT \t LABEL \t ASPECT-START-INDEX \t ASPECT-END-INDEX")
print(train[0])
print(train[1])
print(train[2])
print(train[3])
print(train[4])

SENTENCE 	 ASPECT 	 LABEL 	 ASPECT-START-INDEX 	 ASPECT-END-INDEX
['the decor is not special at all but their food and amazing prices make up for it.', 'decor', 'negative', '4', '9']
['the decor is not special at all but their food and amazing prices make up for it.', 'food', 'positive', '42', '46']
['the decor is not special at all but their food and amazing prices make up for it.', 'prices', 'positive', '59', '65']
['when tables opened up, the manager sat another party before us.', 'tables', 'neutral', '5', '11']
['when tables opened up, the manager sat another party before us.', 'manager', 'negative', '27', '34']


In [7]:
# Please write your code to generate the following data
# x_train_review_int
# x_train_review_masks
# x_train_aspect_int
# x_train_aspect_masks

# x_dev_review_int
# x_dev_review_masks
# x_dev_aspect_int
# x_dev_aspect_masks

# x_test_review_int
# x_test_review_masks
# x_test_aspect_int
# x_test_aspect_masks

# your code goes here
def prepare_data(data):
    review_ints, review_masks, aspect_ints, aspect_masks = [], [], [], []
    for entry in data:
        # Tokenize review
        review_text = entry[0]
        review_ids, review_mask, _ = tokenize(review_text, tokenizer)

        # Tokenize aspect
        aspect_text = entry[1]
        aspect_ids, aspect_mask, _ = tokenize(aspect_text, tokenizer)

        review_ints.append(review_ids)
        review_masks.append(review_mask)
        aspect_ints.append(aspect_ids)
        aspect_masks.append(aspect_mask)

    return np.array(review_ints), np.array(review_masks), np.array(aspect_ints), np.array(aspect_masks)

# Prepare training, development, and test sets
x_train_review_int, x_train_review_masks, x_train_aspect_int, x_train_aspect_masks = prepare_data(train)
x_dev_review_int, x_dev_review_masks, x_dev_aspect_int, x_dev_aspect_masks = prepare_data(dev)
x_test_review_int, x_test_review_masks, x_test_aspect_int, x_test_aspect_masks = prepare_data(test)

# You can check the results as follows:
assert len(x_train_aspect_int) == len(train)
assert len(x_train_aspect_masks) == len(x_train_aspect_int)
assert len(x_test_aspect_int) == len(test)
assert len(x_test_aspect_masks) == len(x_test_aspect_int)
print("x_dev_aspect_int[0]:")
print(x_dev_aspect_int[0])
print("x_dev_aspect_masks[0]:")
print(x_dev_aspect_masks[0])
print("x_dev_review_int[0]:")
print(x_dev_review_int[0])
print("x_dev_review_masks[0]:")
print(x_dev_review_masks[0])

x_dev_aspect_int[0]:
[ 101 8974  102    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
x_dev_aspect_masks[0]:
[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
x_dev_review_int[0]:
[  101  2044  1037  323

Using the BERT `tokenize` function above, we can turn text and aspect words to integers:


We one-hot encode the labels, using 4 (Binary:100) to represent "positive", 2 (Binary:010) for "neutral", and 1 (Binary:001) for "negative".

In [8]:
def label2int(dataset):
  y = []
  for example in dataset:
    if example[2].lower() == "negative":
      y.append([0,0,1])
    elif example[2].lower() == "neutral":
      y.append([0,1,0])
    else:
      # assert example[2].lower() == "positive"
      y.append([1,0,0])
  return y

y_train = label2int(train)
y_dev = label2int(dev)
y_test = label2int(test)
y_train = np.array(y_train)
y_dev = np.array(y_dev)
y_test = np.array(y_test)

print(y_train[0])
print(y_train[1])
print(y_train[2])
print(y_train[3])
print(y_train[4])

[0 0 1]
[1 0 0]
[1 0 0]
[0 1 0]
[0 0 1]


Now we have almost finished the data preprocessing. Unlike in the previous labs, there are two model inputs: review and aspect. The easiest way is to combine the review and aspect into one sentence and then input it into the model.

In [9]:
# Please write your code to combine sentences and aspect words into the following varibles

# x_train_int
# x_train_masks
# x_dev_int
# x_dev_masks
# x_test_int
# x_test_masks

# Tips:
# 1) Use the special token [SEP] to concatenate sentences and aspect words
# 2) Make sure they are paded/truncated to a max length

# your code goes here

def prepare_combined_data(data, tokenizer, max_length=128):
    input_ids, attention_masks = [], []

    for entry in data:
        # Combining sentence and aspect with [SEP] token
        sentence, aspect = entry[0], entry[1]
        combined_text = sentence + " [SEP] " + aspect

        # Tokenizing and encoding the combined text
        tokens = tokenizer.encode_plus(combined_text, add_special_tokens=True, max_length=max_length,
                                       pad_to_max_length=True, return_attention_mask=True, truncation=True)

        input_ids.append(tokens['input_ids'])
        attention_masks.append(tokens['attention_mask'])

    return input_ids, attention_masks

max_length = 128

x_train_int, x_train_masks = prepare_combined_data(train, tokenizer, max_length=max_length)
x_dev_int, x_dev_masks = prepare_combined_data(dev, tokenizer, max_length=max_length)
x_test_int, x_test_masks = prepare_combined_data(test, tokenizer, max_length=max_length)

# Don't forget the to use the np.array function to wrap the outputs
x_train_int_np = np.array(x_train_int)
x_train_masks_np = np.array(x_train_masks)
x_dev_int_np = np.array(x_dev_int)
x_dev_masks_np = np.array(x_dev_masks)
x_test_int_np = np.array(x_test_int)
x_test_masks_np = np.array(x_test_masks)


print(x_dev_int[0])
print(x_dev_masks[0],'\n')
print(x_dev_int_np[0])
print(x_dev_masks_np[0]) # sentence + aspect


[101, 2044, 1037, 3232, 1997, 8974, 1010, 1996, 18726, 1011, 1011, 1045, 2066, 1996, 27940, 1013, 24792, 2621, 4897, 1998, 1996, 13675, 11514, 6508, 26852, 1011, 1011, 2175, 2091, 2307, 1012, 102, 8974, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 

[  101  2044  1037  3232  1997  8974  1010  1996 18726  1011  1011  1045
  2066  1996 27940  1013 24792  2621  4897  1998  

## Model 1: Prebuilt Sequence Classification



The Huggingface transformer package provides many prebuilt models. Now let us try a sequence classification model based on DistilBERT.

The models with BERT are much bigger than our previous models. To run it faster, we can use TPU. Detailed guidelines on how to use TPU can be found from https://www.tensorflow.org/guide/tpu.

In [10]:
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig
import tensorflow as tf

distil_bert = 'distilbert-base-uncased'

config = DistilBertConfig(num_labels=3)
config.output_hidden_states = False

def create_TFDistilBertForSequenceClassification():
  transformer_model = TFDistilBertForSequenceClassification.from_pretrained(distil_bert, config = config)
  input_ids = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
  input_masks_ids = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32')
  X = transformer_model(input_ids, input_masks_ids)
  return tf.keras.Model(inputs=[input_ids, input_masks_ids], outputs = X)

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)

  # Create model on TPU:
  with strategy.scope():
    model = create_TFDistilBertForSequenceClassification()
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model = create_TFDistilBertForSequenceClassification()
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tf_distil_bert_for_sequence_cl  TFSequenceClassifie  66955779   ['input_token[0][0]',            
 assification (TFDistilBertForS  rOutput(loss=None,               'masked_token[0][0]']           
 equenceClassification)         logits=(None, 3),                                                 
                                 hidden_states=None                                           

In [12]:
history = model.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30
22/22 [==============================] - 100s 2s/step - loss: 0.6788 - accuracy: 0.4443 - val_loss: 0.5822 - val_accuracy: 0.4595
Epoch 2/30
22/22 [==============================] - 5s 241ms/step - loss: 0.5675 - accuracy: 0.5110 - val_loss: 0.5260 - val_accuracy: 0.6066
Epoch 3/30
22/22 [==============================] - 5s 229ms/step - loss: 0.5231 - accuracy: 0.6159 - val_loss: 0.5252 - val_accuracy: 0.6306
Epoch 4/30
22/22 [==============================] - 5s 231ms/step - loss: 0.4912 - accuracy: 0.6616 - val_loss: 0.4445 - val_accuracy: 0.7110
Epoch 5/30
22/22 [==============================] - 5s 233ms/step - loss: 0.4169 - accuracy: 0.7397 - val_loss: 0.4385 - val_accuracy: 0.7110
Epoch 6/30
22/22 [==============================] - 5s 231ms/step - loss: 0.4539 - accuracy: 0.6974 - val_loss: 0.4712 - val_accuracy: 0.7402
Epoch 7/30
22/22 [==============================] - 5s 232ms/step - loss: 0.4099 - accuracy: 0.7610 - val_loss: 0.4543 - val_accuracy: 0.7590
Epoch 8

In [13]:
results = model.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

42/42 [==============================] - 7s 83ms/step - loss: 0.6018 - accuracy: 0.7672
[0.6018344759941101, 0.7672155499458313]



## Model 2: Neural bag of words using BERT

Now we want to build the neural network model. We  are going to have a hidden layer with 16 hidden units.

First, we want to transform each index to a BERT embedded vector and then average all vectors to a single one. It has been showed that unweighted average of word vectors outperforms many complicated networks that model semantic and syntactic compositionality. As an example you can take a look at this: (http://anthology.aclweb.org/P/P15/P15-1162.pdf)

To average we need to ignore padded zeros:

In [14]:
class GlobalAveragePooling1DMasked(GlobalAveragePooling1D):
    def call(self, x, mask=None):
        if mask != None:
            return K.sum(x, axis=1) / K.sum(mask, axis=1)
        else:
            return super().call(x)

In [15]:
from transformers import TFDistilBertModel, DistilBertConfig

def get_BERT_layer():
  distil_bert = 'distilbert-base-uncased'
  config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
  config.output_hidden_states = False
  return TFDistilBertModel.from_pretrained(distil_bert, config = config)


In [21]:
import tensorflow as tf
hdepth=16
MAX_SEQUENCE_LENGTH = 128
EMBED_SIZE=100

def create_bag_of_words_BERT():
  # Defining input layers for the BERT model, one for input IDs and one for attention masks
  input_ids_in = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='input_ids')
  input_masks_in = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='input_masks')

  # Initialising the BERT layer from the predefined function
  bert_layer = get_BERT_layer()
  # Getting the BERT model's output using the input IDs and masks
  bert_output = bert_layer(input_ids_in, attention_mask=input_masks_in)[0]

  # Applying a global average pooling layer
  pooled_sent = GlobalAveragePooling1DMasked()(bert_output)

  # Adding a dense (fully connected) layer for hidden representation
  hidden_output = Dense(hdepth, activation='relu', kernel_initializer='glorot_uniform')(pooled_sent)
  # Adding a final dense layer with softmax activation to output probabilities for three classes
  label = Dense(3, activation='softmax', kernel_initializer='glorot_uniform')(hidden_output)

  return Model(inputs=[input_ids_in, input_masks_in], outputs=label, name='Model2_BERT')

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)

  # Create model
  with strategy.scope():
    model2 = create_bag_of_words_BERT()
    optimizer2 = tf.keras.optimizers.Adam(learning_rate=5e-5)
    model2.compile(optimizer=optimizer2, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model2 = create_bag_of_words_BERT()
  model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [22]:
model2.summary()

Model: "Model2_BERT"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 tf_distil_bert_model_1 (TFDist  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 ilBertModel)                   ast_hidden_state=(N               'input_masks[0][0]']            
                                one, 128, 768),                                                   
                                 hidden_states=None                                     

In [23]:
history = model2.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30
22/22 [==============================] - 108s 2s/step - loss: 0.6060 - accuracy: 0.4581 - val_loss: 0.5571 - val_accuracy: 0.5541
Epoch 2/30
22/22 [==============================] - 5s 230ms/step - loss: 0.4825 - accuracy: 0.6300 - val_loss: 0.4152 - val_accuracy: 0.7170
Epoch 3/30
22/22 [==============================] - 5s 232ms/step - loss: 0.3660 - accuracy: 0.7519 - val_loss: 0.3209 - val_accuracy: 0.7838
Epoch 4/30
22/22 [==============================] - 5s 232ms/step - loss: 0.2903 - accuracy: 0.8114 - val_loss: 0.3136 - val_accuracy: 0.7988
Epoch 5/30
22/22 [==============================] - 5s 231ms/step - loss: 0.2333 - accuracy: 0.8571 - val_loss: 0.3105 - val_accuracy: 0.8161
Epoch 6/30
22/22 [==============================] - 5s 233ms/step - loss: 0.1936 - accuracy: 0.8823 - val_loss: 0.3267 - val_accuracy: 0.8228
Epoch 7/30
22/22 [==============================] - 5s 231ms/step - loss: 0.1572 - accuracy: 0.9093 - val_loss: 0.3370 - val_accuracy: 0.8198
Epoch 8

In [24]:
results = model2.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

42/42 [==============================] - 7s 85ms/step - loss: 0.6051 - accuracy: 0.8368
[0.6051443219184875, 0.8368263244628906]


## Model 3: CNN or LSTM with BERT

Please follow the architecture for model2 to construct a CNN or an LSTM model on the top of BERT.

In [26]:
# your code goes here
def create_CNN_BERT():
    input_ids_in = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='input_ids')
    input_masks_in = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='input_masks')

    bert_layer = get_BERT_layer()
    bert_output = bert_layer(input_ids_in, attention_mask=input_masks_in)[0]

    # CNN layer
    conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(bert_output)
    pooled_output = GlobalAveragePooling1D()(conv_layer)

    hidden_output = Dense(hdepth, activation='relu', kernel_initializer='glorot_uniform')(pooled_output)
    label = Dense(3, activation='softmax', kernel_initializer='glorot_uniform')(hidden_output)

    model3 = Model(inputs=[input_ids_in, input_masks_in], outputs=label, name='CNN_BERT_Model')
    return model3

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)

  # Create model
  with strategy.scope():
    model3 = create_CNN_BERT()
    optimizer3 = tf.keras.optimizers.Adam(learning_rate=5e-5)
    model3.compile(optimizer=optimizer3, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model3 = create_CNN_BERT()
  model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [27]:
model3.summary()

Model: "CNN_BERT_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 tf_distil_bert_model_3 (TFDist  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 ilBertModel)                   ast_hidden_state=(N               'input_masks[0][0]']            
                                one, 128, 768),                                                   
                                 hidden_states=None                                  

In [28]:
history = model3.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30
22/22 [==============================] - 108s 2s/step - loss: 0.5951 - accuracy: 0.4794 - val_loss: 0.5255 - val_accuracy: 0.6141
Epoch 2/30
22/22 [==============================] - 5s 234ms/step - loss: 0.4587 - accuracy: 0.6657 - val_loss: 0.3597 - val_accuracy: 0.7628
Epoch 3/30
22/22 [==============================] - 5s 236ms/step - loss: 0.3301 - accuracy: 0.7842 - val_loss: 0.3147 - val_accuracy: 0.8071
Epoch 4/30
22/22 [==============================] - 5s 232ms/step - loss: 0.2751 - accuracy: 0.8235 - val_loss: 0.3133 - val_accuracy: 0.8093
Epoch 5/30
22/22 [==============================] - 7s 302ms/step - loss: 0.2225 - accuracy: 0.8671 - val_loss: 0.3228 - val_accuracy: 0.8078
Epoch 6/30
22/22 [==============================] - 5s 236ms/step - loss: 0.1800 - accuracy: 0.8949 - val_loss: 0.3304 - val_accuracy: 0.8123
Epoch 7/30
22/22 [==============================] - 5s 236ms/step - loss: 0.1439 - accuracy: 0.9206 - val_loss: 0.3666 - val_accuracy: 0.8146
Epoch 8

In [29]:
results = model3.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

42/42 [==============================] - 7s 87ms/step - loss: 0.6101 - accuracy: 0.8421
[0.6100521683692932, 0.8420658707618713]
